# Multi-agent Examples

In this example I show an example of an agent that writes a story using a series of steps.

1. The user provides an idea for their story.
2. The `Outline Bot` generates an outline for the story including section titles and descriptions.
3. The first section content is created by the `Story Bot` from section title/description and the overall story description.
4. The `Summary Bot` creates a summary of the newly generated chapter.
5. The section summary is passed to the `Story Bot` along with section title/description to generate the next section.
6. All sections follow steps 3-5.
...
7. The sections are combined into a full story.

![Story bot design diagram](https://storage.googleapis.com/public_data_09324832787/story_bot_design.svg)

In [1]:
import sys
sys.path.append('..')

import simplechatbot
from simplechatbot.openai import OpenAIChatBot

First I create a new chatbot from OpenAI using the API stored in the keychain file.

I also create a new function `stream_it` that prints the result from the LLM as it is received and returns the full result of the LLM call.

In [2]:
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')
base_chatbot = OpenAIChatBot.new(
    model_name = 'gpt-4o-mini', 
    api_key=keychain['openai'],
)

def stream_it(chatbot: simplechatbot.ChatBot, msg: str | None = None) -> simplechatbot.ChatResult:
    stream = chatbot.chat_stream(msg, add_to_history=False)
    for chunk in stream:
        print(chunk.content, end='', flush=True)
    return stream.collect()

Next we need to direct the LLM to create an outline for the story. I do this by creating a system prompt describing the task of creating an outline and a pydantic class to direct the LLM on how to structure its response. This is important because the output of the outline bot will be used for generating chapters.

In [3]:
import pydantic

from typing import Optional

#from pydantic import BaseModel, Field
import pydantic

class StoryOutline(pydantic.BaseModel):
    """Outline of the story."""

    story_topic: str = pydantic.Field(description="The topic of the story.")

    part1_title: str = pydantic.Field(description="Title of Part 1 of the story.")
    part1_description: str = pydantic.Field(description="Longer description of part 1.")

    part2_title: str = pydantic.Field(description="Title of Part 2 of the story.")
    part2_description: str = pydantic.Field(description="Longer description of part 2.")

    part3_title: str = pydantic.Field(description="Title of Part 3 of the story.")
    part3_description: str = pydantic.Field(description="Longer description of part 3.")

    def outline_str(self) -> str:
        return f'topic: {self.story_topic}\n\n{self.part1_title}: {self.part1_description}\n\n{self.part2_title}: {self.part2_description}\n\n{self.part3_title}: {self.part3_description}'


system_prompt = '''
The user will provide you a description of a story, and you must create a chapter outline with titles and brief descriptions.
Each section should contain a title and a brief description of what happens in that section.
The sections should all be part of a single narrative ark, but each section should have a complete beginning, middle, and end.
'''
outline_bot = base_chatbot.structbot_from_model(
    output_structure=StoryOutline,
    system_prompt=system_prompt,
)

q = f'Write an outline for a story about two friends who met when they were young and then lost touch. They meet again as adults and have to navigate their new relationship.'
outline: StoryOutline = outline_bot._model.with_structured_output(StoryOutline).invoke(q)
print(outline.outline_str())

topic: Rekindling Friendship

The Innocent Beginning: Two children, Emma and Jack, meet at a summer camp and quickly form a deep and meaningful friendship. They spend their days exploring the woods, sharing secrets, and dreaming about the future. However, as summer ends, their paths diverge, and they lose touch as they go back to their separate lives.

Years Apart: As the years pass, Emma and Jack grow up in different cities, each facing their own challenges. Emma pursues a career in art, while Jack becomes a successful entrepreneur. Despite their busy lives, they occasionally think about each other, reminiscing about their childhood friendship but never reaching out.

Unexpected Reunion: Fate brings Emma and Jack back together at a mutual friend's wedding. Their initial joy is tinged with awkwardness as they realize how much they have changed. They spend the weekend reconnecting, sharing their life stories, and navigating the complexities of their new adult relationship, ultimately de

Next we create a bot that will generate the actual chapter content based on information generated in the outline and a summary of the previous chapter (if one exists).

In [4]:
chapter_bot_system_prompt = f'''
You are designed to write a single chapter of a larger story based on the following information:

+ Overall story topic: The topic of the full story.
+ Chapter title: The title of the section you are writing.
+ Chapter description: A longer description of what happens in the section.
+ (optional) previous chapter summary: A summary of the previous chapter.

Your responses should only include text that is part of the story. Do not include the chapter title 
or any other information that is not part of the story itself.
'''

def chapter_bot_prompt(
    story_topic: str,
    chapter_title: str,
    chapter_description: str,
    previous_chapter_summary: Optional[str] = None,
) -> str:
    return f'''
    General story topic: "{story_topic}"

    Section title: "{chapter_title}"

    Description: "{chapter_description}"

    Previous chapter summary: "{previous_chapter_summary if previous_chapter_summary is not None else 'No previous chapter - this is the first!'}"
    '''

chapter_bot = base_chatbot.empty().clone(
    system_prompt=chapter_bot_system_prompt,
)

Now we actually create the chatper using the prompt.

In [5]:
cb1 = base_chatbot.empty().clone(
    system_prompt=system_prompt,
)
prompt = chapter_bot_prompt(outline.story_topic, outline.part1_title, outline.part1_description)
chapter1 = stream_it(chapter_bot, prompt)

The sun dipped low on the horizon, casting a warm golden hue over Maplewood Park, where the gentle rustle of leaves danced with the laughter of children playing nearby. It was a place that once brimmed with memories for Emma and Jake, two childhood friends who had drifted apart over the years, consumed by the tides of life.

Emma sat on a weathered wooden bench, her sketchbook resting on her lap, the pages blank but for a few light pencil lines that hinted at her longing to capture the beauty around her. She had come to the park to find inspiration, but what she found instead was a wave of nostalgia that threatened to engulf her. She remembered the countless afternoons spent here, sketching the trees and laughing until their sides hurt.

As she drew absentmindedly, a familiar voice broke through her reverie. “Is that you, Emma?” 

Startled, she looked up to find Jake standing a few paces away, his hair tousled and a bright smile lighting up his face. He had changed since the last time 

Next I create a bot that will summarize a chapter. We will eventually feed this into the creation of chapter 2.

In [ ]:
summary_bot_system_prompt = f'''
You need to create a summary of the story chapter provided to you by the user.
The summary should include names of relevant characters and capture the story arc of the chapter.
'''
summary_bot = base_chatbot.empty().clone(
    system_prompt=summary_bot_system_prompt,
)
ch1_summary = stream_it(summary_bot, f'Chapter text:\n\n{chapter1.content}')

In this chapter, the setting is Maplewood Park, where Emma, a passionate artist, seeks inspiration but instead finds herself engulfed in nostalgia. While sitting on a bench, she unexpectedly reunites with her childhood friend, Jake. They share a warm exchange filled with memories of their past, including a humorous recollection of a failed treehouse business they attempted to start in their youth. Their conversation gradually bridges the gap created by years of separation, revealing their mutual longing for the connection they once had.

As they reminisce, both characters express how much they've missed each other, leading to a rekindling of their friendship. The chapter captures the idyllic atmosphere of the park, filled with laughter and memories, and concludes with a hopeful pact between Emma and Jake to return to the park more often, indicating their desire to not let life interfere with their bond again. The chapter beautifully illustrates the themes of nostalgia, reconnection, an

Now we actually generate the second chapter using the outline and the previous chapter summary.

In [8]:
prompt = chapter_bot_prompt(
    outline.story_topic,
    outline.part2_title,
    outline.part2_description,
    ch1_summary.content,
)
chapter2 = stream_it(chapter_bot, prompt)

The sun dipped lower in the sky, casting a warm golden hue over Maplewood Park. The sounds of laughter and the rustle of leaves created a serene backdrop as Emma and Jake settled into the rhythm of their conversation. Time seemed to melt away as they navigated through a labyrinth of memories, each twist and turn revealing shared secrets and unspoken words.

“I can’t believe we actually thought we could build that treehouse,” Jake chuckled, shaking his head. “I remember us drawing out blueprints on the back of that cereal box.”

Emma laughed, her eyes sparkling with joy. “And we had the audacity to charge the neighborhood kids for entry! I think we made a whole twenty cents before it collapsed.”

“Twenty cents we earned through sheer ambition,” Jake replied, grinning. “I still say it was a brilliant business model.”

Their laughter echoed through the park, and for a moment, it felt as if the years melted away. The weight of their separate lives—filled with college, jobs, and new friends

Create a summary for chapter 2 now.

In [9]:
ch2_summary = stream_it(summary_bot, f'Previous chapter text:\n\n{chapter1.content}')

In this chapter, Emma and Jake, childhood friends who have grown apart, unexpectedly reunite at Maplewood Park, a place filled with their shared memories. Emma, seeking inspiration for her art, finds herself reminiscing about their past when Jake approaches her, sparking a conversation filled with nostalgia and warmth. They share stories of their childhood adventures, including a humorous attempt to build a treehouse, which brings laughter and eases the tension between them.

As they reconnect, both express how much they've missed each other, realizing the significance of their friendship that time had frayed. The atmosphere shifts from a sense of loss to a hopeful promise of rekindling their bond. Jake proposes that they return to the park more often, and Emma eagerly agrees, indicating a desire to not let life interfere with their relationship again. The chapter concludes with them leaving the park together, laughter in the air, symbolizing the first steps toward rebuilding their fri

Use the summary of chapter 2 and outline information to create chapter 3.

In [10]:
prompt = chapter_bot_prompt(
    outline.story_topic,
    outline.part2_title,
    outline.part2_description,
    ch2_summary.content,
)
chapter3 = stream_it(chapter_bot, prompt)

The following weeks felt like a dream to Emma. Each time she returned to Maplewood Park, the familiar sights and sounds wrapped around her like a warm blanket. The vibrant greens of the leaves, the laughter of children echoing in the distance, and the sweet scent of blooming flowers all reminded her of the happiness she had shared with Jake. It was as if time had folded in on itself, allowing them to reclaim a part of their lives that had been set aside for far too long.

Their meetups became a ritual. They would stroll through the park, sharing stories about their lives since they had last seen each other. Jake spoke about his job at the local theater, his passion for acting igniting with every word. Emma found herself captivated, her heart swelling with pride for her friend. She shared her struggles as an artist, the challenges of finding her voice and the pressure to succeed. Jake listened intently, offering encouragement that felt like a lifeline.

One sunny afternoon, as they sat 

And now we can review the full text of the book.

In [13]:
story = f'''

Overview: {outline.story_topic}

== Chapter 1: {outline.part1_title} ==

{chapter1.content}


== Chapter 2: {outline.part2_title} ==

{chapter2.content}


== Chapter 3: {outline.part3_title} ==

{chapter3.content}

'''

import pprint
pprint.pprint(story)

('\n'
 '\n'
 'Overview: Rekindling Friendship\n'
 '\n'
 '== Chapter 1: The Innocent Beginning ==\n'
 '\n'
 'The sun dipped low on the horizon, casting a warm golden hue over Maplewood '
 'Park, where the gentle rustle of leaves danced with the laughter of children '
 'playing nearby. It was a place that once brimmed with memories for Emma and '
 'Jake, two childhood friends who had drifted apart over the years, consumed '
 'by the tides of life.\n'
 '\n'
 'Emma sat on a weathered wooden bench, her sketchbook resting on her lap, the '
 'pages blank but for a few light pencil lines that hinted at her longing to '
 'capture the beauty around her. She had come to the park to find inspiration, '
 'but what she found instead was a wave of nostalgia that threatened to engulf '
 'her. She remembered the countless afternoons spent here, sketching the trees '
 'and laughing until their sides hurt.\n'
 '\n'
 'As she drew absentmindedly, a familiar voice broke through her reverie. “Is '
 'that you